In [16]:
import tensorflow as tf 
import tensorflow.contrib.rnn as rnn
import numpy as np
import os,os.path
from data_pb2 import Sequence
from prep_data import prep_data
from moveit_msgs.msg import RobotState
from sensor_msgs.msg import JointState
import sys
from data_generation import call_compute_fk_service,random_valid_end_point_generator,get_poses
from control_msgs.msg import FollowJointTrajectoryAction, FollowJointTrajectoryGoal
from trajectory_msgs.msg import JointTrajectoryPoint
from moveit_msgs.msg import RobotState
from sensor_msgs.msg import JointState
import copy
import rospy
import moveit_msgs.msg
import actionlib
import moveit_commander
import geometry_msgs.msg
from geometry_msgs.msg import Pose
import math
import random
from moveit_msgs.srv import GetPositionFK
from moveit_msgs.srv import GetPositionFKRequest
from moveit_msgs.srv import GetPositionFKResponse
from sensor_msgs.msg import JointState
from moveit_msgs.msg import RobotState
from std_msgs.msg import Header
from data_pb2 import Sequence
import sys,os
from data_generation import random_valid_end_point_generator,call_compute_fk_service    

prime="/home/shashank/catkin_ws/src/rnn_ur5/"
data_home=prime+"dataset/"
file_name = data_home + 'sequence_%05d'
model_ckpt_path = prime+"modelckpt/"
model_events = prime+"/modelevents/"
from sklearn.utils import shuffle

path, dirs, files = next(os.walk(data_home))
num_sequences = len(files)
#print num_sequences

batch_size=64
num_features = 6 ##[desired_pose,current_pose]
output_sequence_length = 6 ##6 joint angles
num_units = 6
samples_per_path=100

inputs=[]
labels=[]
initial_states=[]
temp_inputs=[]

inp=[]
out=[]

data=Sequence()

for i in range(num_sequences):
    data.Clear()
    f=open(file_name%i,"rb")
    data.ParseFromString(f.read())
    for i in range(len(data.xyz)-1):
        x=[]
        y=[]
        [x.append(j) for j in data.xyz[i].poses]
        [x.append(j) for j in data.xyz[i+1].poses]
        [x.append(j/np.pi) for j in data.thetas[i].theta_values]
        [y.append(j) for j in data.thetas[i+1].theta_values]
        inp.append(x)
        out.append(y)

    f.close()

inp=np.array(inp)
out=np.array(out)
out=out/np.pi

ckpt_start=0

# for i in np.arange(0,len(out),1):
# 	print '~~~~~~'
# 	print out[i]
# 	print '\n\n'
# 	print inp[i]
# 	print '~~~~~~'
# # In[89]:


import tensorflow as tf


# In[90]:


input_dimension=12
output_dimension=6

inputs = tf.placeholder(shape=[None,input_dimension],dtype=tf.float32,name='INPUT')
labels = tf.placeholder(shape=[None,output_dimension],dtype=tf.float32,name='OUTPUT')

print "\n\n\n\n"
print np.shape(inp)
print np.shape(out)

units = [30,50,6]

# In[92]:




with tf.name_scope('Dense_1'):
	fc1 = tf.layers.dense(inputs=inputs,units=units[0],use_bias=False,activation=tf.nn.sigmoid)

# with tf.name_scope('Dense_2'):
# 	fc2=tf.layers.dense(inputs=fc1,units=units[1],use_bias=False,activation=tf.nn.tanh)

# with tf.name_scope('Dense_3'):	
# 	fc3=tf.layers.dense(inputs=fc2,units=units[2],use_bias=False,activation=tf.nn.tanh)    

with tf.name_scope('Dense_4'):	
	fc4=tf.layers.dense(inputs=fc1,units=units[1],use_bias=False,activation=tf.nn.sigmoid)    

with tf.name_scope('Output'):
	output=tf.layers.dense(inputs=fc4,units=units[2],activation=tf.nn.tanh,use_bias=False)

with tf.name_scope('LOSS'):
        loss=tf.reduce_sum(tf.losses.mean_squared_error(labels=labels,predictions=output))
    
optim = tf.train.AdamOptimizer(name="Optimizer",learning_rate=1e-5).minimize(loss)

loss_summary = tf.summary.scalar('loss',loss)
saver = tf.train.Saver()

# In[96]:
write = 0
leng=len(inp)
train_size = int(0.8*leng)
train_x = inp[0:train_size,:]
train_y = out[0:train_size,:]
test_x  = inp[train_size:,:]
test_y  = out[train_size:,:]






(184749, 12)
(184749, 6)


In [18]:
global group_arm,current_joint_values
current_joint_values = JointState()
moveit_commander.roscpp_initializer.roscpp_initialize(sys.argv)
rospy.init_node('main_node', anonymous=True)
robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()
group_arm = moveit_commander.MoveGroupCommander("manipulator")
group_arm.set_planner_id("RRTConnectkConfig")

waypoints=[]
initial_ee_pose = get_poses(False,group_arm)
initial_joint_angles = get_poses(True,group_arm)
waypoints.append(initial_ee_pose)
final_ee_pose = geometry_msgs.msg.Pose()
(plan,fraction,final_ee_pose)=random_valid_end_point_generator(waypoints,group_arm,initial_ee_pose)
intermediate_ee_poses = []
path_length = len(plan.joint_trajectory.points)
intermediate_joint_values = [plan.joint_trajectory.points[j].positions for j in range(path_length)]
[intermediate_ee_poses.append(call_compute_fk_service(joints)) for joints in intermediate_joint_values]   

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [27]:
intermediate_ee_poses[0]

x: 0.817571329176
y: 0.191181544486
z: 0.508030301952

In [28]:
x = []
[x.append([i.x,i.y,i.z]) for i in intermediate_ee_poses]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [30]:
np.shape(x)

(87, 3)

In [31]:
x_1 = np.roll(x,-1,0)

[[0.8175713291759676, 0.1911815444863543, 0.5080303019519539],
 [0.8175713291759676, 0.1911815444863543, 0.5080303019519539],
 [0.739870346349236, 0.44015878045413925, 0.6373352329516172],
 [0.656918662844988, 0.5664588762330349, 0.716439323130818],
 [0.6384985570477326, 0.5769588046026609, 0.7244348620057882],
 [0.5814980779722404, 0.6320672533486227, 0.7663160664783898],
 [0.5293585234406102, 0.6718746506869706, 0.8011387577330428],
 [0.4796182518595655, 0.7027269516280793, 0.8323841537446818],
 [0.43172844751300565, 0.7269076989880747, 0.8611577118608273],
 [0.38551215583091974, 0.7456962418016636, 0.8880150858349587],
 [0.34088181317754324, 0.7599583209063578, 0.9133093621859317],
 [0.2977872275014878, 0.7703284474948017, 0.9372859376809366],
 [0.2561964783841294, 0.7772959917983187, 0.9601250843845713],
 [0.2160875246857703, 0.7812523827816515, 0.9819644018603347],
 [0.17743924927966193, 0.7825201861772748, 1.0029155507437164],
 [0.14025296893335487, 0.7813681304676409, 1.02305447

In [33]:
x_1

array([[ 8.17571329e-01,  1.91181544e-01,  5.08030302e-01],
       [ 7.39870346e-01,  4.40158780e-01,  6.37335233e-01],
       [ 6.56918663e-01,  5.66458876e-01,  7.16439323e-01],
       [ 6.38498557e-01,  5.76958805e-01,  7.24434862e-01],
       [ 5.81498078e-01,  6.32067253e-01,  7.66316066e-01],
       [ 5.29358523e-01,  6.71874651e-01,  8.01138758e-01],
       [ 4.79618252e-01,  7.02726952e-01,  8.32384154e-01],
       [ 4.31728448e-01,  7.26907699e-01,  8.61157712e-01],
       [ 3.85512156e-01,  7.45696242e-01,  8.88015086e-01],
       [ 3.40881813e-01,  7.59958321e-01,  9.13309362e-01],
       [ 2.97787228e-01,  7.70328447e-01,  9.37285938e-01],
       [ 2.56196478e-01,  7.77295992e-01,  9.60125084e-01],
       [ 2.16087525e-01,  7.81252383e-01,  9.81964402e-01],
       [ 1.77439249e-01,  7.82520186e-01,  1.00291555e+00],
       [ 1.40252969e-01,  7.81368130e-01,  1.02305447e+00],
       [ 1.04500802e-01,  7.78029916e-01,  1.04246494e+00],
       [ 7.01858273e-02,  7.72706325e-01